#Instructor:
- Sarawoot Kongyoung
- Piyawat Chuangkrud

References:

[PyTerrier’s documentation](https://pyterrier.readthedocs.io/en/latest/)

[Notebooks](https://github.com/terrier-org/pyterrier/blob/master/examples/notebooks.md)

## Link for Notebook
**https://shorturl.at/psAPQ**

## Link for Slide
**https://shorturl.at/ouzH0**

# Prerequisites
You will need PyTerrier installed. PyTerrier also needs Java to be installed, and will find most installations.


In [1]:
!pip install python-terrier
!pip install datasets
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.2 MB/s eta 0:00:00
  Preparing metadata (setup

# Import Libraries

In [2]:
import pyterrier as pt
from pyterrier.measures import *
from datasets import load_dataset
from pythainlp import word_tokenize
import pandas as pd
import re

You must run pt.init() before other pyterrier functions and classes.

In [3]:
import pyterrier as pt
if not pt.started():
    pt.init()

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



# Load Dataset
We're going to use a very old IR test collection called [Mr. TyDi](https://huggingface.co/datasets/castorini/mr-tydi) . This is a multi-lingual benchmark dataset built on TyDi, covering eleven typologically diverse languages. It is designed for monolingual retrieval, specifically to evaluate ranking with learned dense representations.

## Load Corpus for Index

In [4]:
corpus = load_dataset('castorini/mr-tydi-corpus', 'thai')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for castorini/mr-tydi-corpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/castorini/mr-tydi-corpus
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `dat

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
corpus

DatasetDict({
    train: Dataset({
        features: ['docid', 'title', 'text'],
        num_rows: 568855
    })
})

In [9]:
corpus['train'][0]

{'docid': '1#0',
 'title': 'หน้าหลัก',
 'text': 'วิกิพีเดียดำเนินการโดยมูลนิธิวิกิมีเดีย องค์กรไม่แสวงผลกำไร ผู้ดำเนินการอีกหลาย ได้แก่'}

## Load Dataset for create topics & qrels.

In [6]:
dataset = load_dataset('castorini/mr-tydi', 'thai')

Generating train split:   0%|          | 0/3319 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/807 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1190 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 3319
    })
    dev: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 807
    })
    test: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 1190
    })
})

In [10]:
dataset['test'][0]

{'query_id': '4131',
 'query': 'อยุธยามีกี่อำเภอ ?',
 'positive_passages': [{'docid': '4203#14', 'text': '', 'title': ''}],
 'negative_passages': []}

In [11]:
df = corpus['train'].to_pandas()

In [12]:
df

,docid,title,text
0,1#0,หน้าหลัก,วิกิพีเดียดำเนินการโดยมูลนิธิวิกิมีเดีย องค์กร...
1,545#0,ดาราศาสตร์,ดาราศาสตร์ คือวิชาวิทยาศาสตร์ที่ศึกษาวัตถุท้อง...
2,545#1,ดาราศาสตร์,ดาราศาสตร์เป็นหนึ่งในสาขาของวิทยาศาสตร์ที่เก่า...
3,545#2,ดาราศาสตร์,การค้นพบสิ่งต่าง ๆ ในเรื่องของดาราศาสตร์ที่เผย...
4,545#3,ดาราศาสตร์,ไม่ควรสับสนระหว่างดาราศาสตร์โบราณกับโหราศาสตร์...
...,...,...,...
568850,1001584#1,วัดเกยไชยเหนือ (บรมธาตุ),จากหลักฐานที่กล่าวมาข้างต้น ทำให้เชื่อได้ว่า ว...
568851,1001591#0,มาเลยาเซมเลีย,"มาเลยาเซมเลีย ( แปล: ""แผ่นดินน้อย"") เป็นเนินเข..."
568852,1001591#1,มาเลยาเซมเลีย,แหลมมืยซ์ฮาโคยังคงเกี่ยวข้องกับการยืนยัดสู้อัน...
568853,1001591#2,มาเลยาเซมเลีย,กองกำลังนาวิกโยธินยึดพื้นที่คืนได้ ผู้บัญชาการ...


# Indexing a Corpus

In [8]:
def iterdf():
    for data in corpus['train']:
        docno = data['docid']
        title = data['title']
        text = data['text']
        text = title + ' ' + text
        yield {'docno':docno, 'text': ' '.join(word_tokenize(text, engine="newmm"))}

Use this Indexer if you wish to index an iter of dicts (possibly with multiple fields). This version is optimized by using multiple threads and POSIX fifos to tranfer data, which ends up being much faster.

In [13]:
index_path = './tydi-index-corpus'
indexer = pt.IterDictIndexer(index_path,overwrite=True, stemmer=None, stopwords=None, tokeniser="UTFTokeniser")

In [14]:
%%time
itix = indexer.index(iterdf())

CPU times: user 14min 13s, sys: 7.6 s, total: 14min 20s
Wall time: 11min 18s


# Retrieval
BatchRetrieve is one of the most commonly used PyTerrier objects. It represents a retrieval transformation, in which queries are mapped to retrieved documents. BatchRetrieve uses a pre-existing Terrier index data structure, typically saved on disk.

In [15]:
def es_preprocess(text):
    text = ' '.join(word_tokenize(text))
    return text

In [51]:
%%time
tfidf_nostem = pt.apply.query(
    lambda row: es_preprocess(row.query)
    ) >> pt.BatchRetrieve(itix, wmodel='TF_IDF')

CPU times: user 6.34 ms, sys: 0 ns, total: 6.34 ms
Wall time: 7.08 ms


In [56]:
%%time
tfidf_nostem2 = pt.apply.query(
    lambda row: es_preprocess(row.query)
    ) >> pt.BatchRetrieve(itix, wmodel="BM25")

CPU times: user 7.42 ms, sys: 8 µs, total: 7.42 ms
Wall time: 5.94 ms


In [57]:
%%time
tfidf_nostem3 = pt.apply.query(
    lambda row: es_preprocess(row.query)
    ) >> pt.BatchRetrieve(itix, wmodel="PL2")

CPU times: user 4.7 ms, sys: 1.01 ms, total: 5.7 ms
Wall time: 5.16 ms


In [17]:
def cleanstr(text):
    text = text.replace('?', '')
    text = re.sub(r'[^\u0E00-\u0E7Fa-zA-Z0-9 ]', '', text)

    return text

## Create Topics

In [18]:
topics = []
for data in dataset['test']:
    query = data['query']
    qid = data['query_id']
    tmp = {'qid':qid, 'query':cleanstr(query)}
    topics.append(tmp)

In [19]:
topics = pd.DataFrame(topics)

In [20]:
topics

,qid,query
0,4131,อยุธยามีกี่อำเภอ
1,4132,ใครเป็นผู้ก่อการ ใน เหตุโจมตีในนอร์เวย์ พศ 2554
2,4133,มาสค์ไรเดอร์เดนโอ ออกอากาศครั้งแรกเมื่อไหร่
3,4134,สกอตแลนด์ มีเมืองหลวงชื่ออะไร
4,4135,แก๊สเรือนกระจกมีส่วนประกอบของก๊าซใดเป็นหลัก
...,...,...
1185,5317,ชามด์เป็นเรื่องราวเกี่ยวกับแม่มดใช่หรือไม่
1186,5318,ใครเป็นนักแสดงนำหญิงในเรื่อง เพราะมีเธอ
1187,5319,พม่ามีกี่จังหวัด
1188,5320,เอ็มทีวีในเอเชียเริ่มมีครั้งแรกในปี พศใด


## Create Qrels

In [21]:
qrels = []
for row in dataset['test']:
    for p in row['positive_passages']:
        qrels.append({
            'qid' : row['query_id'],
            'docno' : p['docid'],
            'label' : 1,
            'iter' : 1
        })

In [22]:
qrels = pd.DataFrame(qrels)

In [23]:
qrels

,qid,docno,label,iter
0,4131,4203#14,1,1
1,4132,383828#1,1,1
2,4133,70522#0,1,1
3,4134,23316#1,1,1
4,4135,152611#0,1,1
...,...,...,...,...
1363,5317,121948#0,1,1
1364,5318,644211#7,1,1
1365,5319,1000543#0,1,1
1366,5320,18287#1,1,1


# Evaluation

In [59]:
%%time
pt.Experiment(
    [tfidf_nostem,tfidf_nostem2,tfidf_nostem3],
    topics,
    qrels,
    ['map_cut_10', 'recip_rank', nDCG@5],
    names=['TFIDF','BM25','PL2'],
    round=4
)

CPU times: user 4min 55s, sys: 2.36 s, total: 4min 57s
Wall time: 4min 46s


,name,map_cut_10,recip_rank,nDCG@5
0,TFIDF,0.3271,0.3571,0.3502
1,BM25,0.3159,0.3455,0.3396
2,PL2,0.2972,0.3253,0.3182


# Search

In [40]:
df = corpus['train'].to_pandas()

In [41]:
dataset['test'][2]

{'query_id': '4133',
 'query': 'มาสค์ไรเดอร์เดนโอ ออกอากาศครั้งแรกเมื่อไหร่ ?',
 'positive_passages': [{'docid': '70522#0', 'text': '', 'title': ''}],
 'negative_passages': []}

In [42]:
query =' '.join(word_tokenize(cleanstr('อยุธยามีกี่อำเภอ')))

In [43]:
query

'อยุธยา มี กี่ อำเภอ'

In [44]:
out = tfidf_nostem.search(query)[:10]
out

,qid,docid,docno,rank,score,query_0,query
0,1,5153,2358#4,0,13.248834,อยุธยา มี กี่ อำเภอ,อยุธยา มี กี่ อำเภอ
1,1,174377,130982#3,1,12.994968,อยุธยา มี กี่ อำเภอ,อยุธยา มี กี่ อำเภอ
2,1,174375,130982#1,2,12.848158,อยุธยา มี กี่ อำเภอ,อยุธยา มี กี่ อำเภอ
3,1,174378,130982#4,3,12.226895,อยุธยา มี กี่ อำเภอ,อยุธยา มี กี่ อำเภอ
4,1,174374,130982#0,4,12.080002,อยุธยา มี กี่ อำเภอ,อยุธยา มี กี่ อำเภอ
5,1,130060,79452#0,5,11.344741,อยุธยา มี กี่ อำเภอ,อยุธยา มี กี่ อำเภอ
6,1,175408,132041#119,6,11.216061,อยุธยา มี กี่ อำเภอ,อยุธยา มี กี่ อำเภอ
7,1,11136,4203#24,7,10.861052,อยุธยา มี กี่ อำเภอ,อยุธยา มี กี่ อำเภอ
8,1,154858,108022#0,8,10.790134,อยุธยา มี กี่ อำเภอ,อยุธยา มี กี่ อำเภอ
9,1,119219,69517#4,9,10.584110,อยุธยา มี กี่ อำเภอ,อยุธยา มี กี่ อำเภอ


In [45]:
for row in out['docno']:
    print(row)
    print(df[df['docid'] == row].text.tolist()[0])
    print('-'*10)

2358#4
ปัจจุบันบริเวณนี้เป็นส่วนหนึ่งของอำเภอพระนครศรีอยุธยา จังหวัดพระนครศรีอยุธยา พื้นที่ที่เคยเป็นเมืองหลวงของไทยนั้น คือ อุทยานประวัติศาสตร์พระนครศรีอยุธยา[6]ตัวนครปัจจุบันถูกตั้งขึ้นใหม่ห่างจากกรุงเก่าไปเพียงไม่กี่กิโลเมตร
----------
130982#3
อำเภอหนองกี่ตั้งอยู่ทางทิศตะวันตกของจังหวัด มีอาณาเขตติดต่อกับเขตการปกครองข้างเคียงดังต่อไปนี้
----------
130982#1
อำเภอหนองกี่ เดิมเป็นพื้นที่อยู่ในเขตการปกครอง อำเภอนางรอง กระทรวงมหาดไทย ประกาศตั้งเป็นกิ่งอำเภอหนองกี่ เมื่อวันที่ 1 มีนาคม 2517 ขณะนั้นแบ่งเขตการปกครองออกเป็น 3 ตำบล ได้แก่ ตำบลหนองกี่ ตำบลเย้ยปราสาท ตำบลหนองไผ่ ต่อมาได้มีพระราชกฤษฎีกายกฐานะขึ้นเป็นอำเภอ เมื่อวันที่ 13 เมษายน 2522
----------
130982#4
อำเภอหนองกี่แบ่งพื้นที่การปกครองออกเป็น 10 ตำบล 108 หมู่บ้าน
ท้องที่อำเภอหนองกี่ประกอบด้วยองค์กรปกครองส่วนท้องถิ่น 11 แห่ง ได้แก่การเดินทาง
ทางแยกต่างระดับ อำเภอสีคิ้ว เลี้ยวมาทางอำเภอโชคชัย ผ่านอำเภอหนองบุญมาก ถึงสี่แยกอำเภอหนองกี่ จากนั้นเลี้ยวซ้ายประมาณ 3 กิโลเมตร ก็จะถึงหาด
----------
130982#0
หนองกี่ เป็นอำเภอหนึ่งของจังหวัดบ

# Assignment

Implement indexing and retrieval systems using more weighting models and evaluation metrics than those used in lab settings. Then, compare the results.